In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

pd.set_option('display.max_rows', None)


In [2]:
item_txt_emb = torch.load('pth/item_txt_emb.pth')

In [3]:
item_txt_emb

{'item_id': array(['11484359_DEAL_GROUP', '12007429_DEAL_GROUP',
        '13925394_DEAL_GROUP', ..., '2934574682_SPU_SHANGOU',
        '1902590009_SPU_SHANGOU', '1130377853_SPU_SHANGOU'], dtype=object),
 'embs': tensor([[ 0.2956,  0.8539, -0.8471,  ...,  0.3531,  0.4254, -0.0499],
         [ 0.1959, -0.1708, -0.7584,  ...,  0.2166,  0.2722, -0.0766],
         [-0.2213,  0.2221, -1.1309,  ...,  0.7301,  0.5090,  0.2455],
         ...,
         [ 0.8176,  0.1025, -0.7570,  ...,  0.7587,  0.2542, -0.0212],
         [ 0.3344, -0.2129,  0.3206,  ...,  0.6285,  0.0940, -0.0119],
         [ 0.1698,  0.4039, -0.0805,  ...,  0.4815,  0.3652, -0.0958]])}

In [4]:
model_file = 'saved/SASRec-Jan-11-2022_17-00-01.pth'

checkpoint = torch.load(model_file)
config = checkpoint['config']
config.final_config_dict['user_inter_num_interval'] = None
# config.final_config_dict['load_col']['inter'].append('cityid')
# config.final_config_dict['unused_col']['inter'] = ['exposure_time', 'exposure_time_list']
# config.final_config_dict['save_dataset'] = True
# config.final_config_dict['save_dataloaders'] = True

init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

11 Jan 17:33    INFO  
General Hyper Parameters:
gpu_id = 3
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 500
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'test_only'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = exposure_time
seq_l

In [12]:
token2id = train_data.dataset.field2token_id[train_data.dataset.iid_field]

In [15]:
%%time
txt_emb = torch.zeros((dataset.item_num, item_txt_emb['embs'].size(-1)))
for item, emb in zip(item_txt_emb['item_id'], item_txt_emb['embs']):
    if item in token2id:
        item_id = token2id[item]
        txt_emb[item_id] = emb
txt_emb

CPU times: user 8.17 s, sys: 251 ms, total: 8.42 s
Wall time: 7.49 s


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.3004, -0.1641, -0.3410,  ...,  0.0922,  0.1433, -0.1395],
        [-0.5276, -0.0614, -0.7724,  ...,  0.3321, -0.0469,  0.2125],
        ...,
        [ 0.0351,  0.2441, -0.2572,  ...,  0.2282, -0.3158, -0.1968],
        [ 0.5718,  0.2839, -1.1755,  ...,  1.0715,  0.1697,  0.1342],
        [ 0.4166, -0.1166, -0.6008,  ...,  0.3087,  0.0053, -0.2990]])

In [13]:
token2id

{'[PAD]': 0,
 '195907074_POI_LVYOU': 1,
 '627460853_POI_WAIMAI': 2,
 '695892448_DEAL_GROUP': 3,
 '90215542_POI_HOTEL2': 4,
 '1512708_POI_HOTEL2': 5,
 '160330165_POI_HOTEL2': 6,
 '189752057_POI_HOTEL2': 7,
 '150562253_POI_HOTEL2': 8,
 '3275383_POI_HOTEL2': 9,
 '156669372_POI_HOTEL2': 10,
 '152577368_POI_HOTEL2': 11,
 '194166857_POI_HOTEL2': 12,
 '170581448_POI_HOTEL2': 13,
 '118519475_POI_HOTEL2': 14,
 '179101859_POI_HOTEL2': 15,
 '98612077_POI_HOTEL2': 16,
 '193911182_POI_WAIMAI': 17,
 '695468207_DEAL_GROUP': 18,
 '1143637118_POI_LVYOU': 19,
 '652086811_DEAL_GROUP': 20,
 '1482629442_POI_WAIMAI': 21,
 '103717195_POI_WAIMAI': 22,
 '164372514_POI_WAIMAI': 23,
 '159274958_POI_HOTEL2': 24,
 '178799555_POI_HOTEL2': 25,
 '1206227_POI_HOTEL2': 26,
 '627297071_DEAL_GROUP': 27,
 '708728018_POI_GROUP': 28,
 '4016306_POI_WAIMAI': 29,
 '1193044965_POI_GROUP': 30,
 '656374293_DEAL_GROUP': 31,
 '1283891511_POI_HOTEL2': 32,
 '1661733303_POI_HOTEL2': 33,
 '694194882_DEAL_GROUP': 34,
 '689970007_DEAL_GR